In [1]:
import azure.ai.ml

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential =  DefaultAzureCredential()
    credential.get_token("https://management.azure.com/default")
except:
    credential = InteractiveBrowserCredential()
    



DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: No token received.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.


In [2]:
ml_client = MLClient(
    credential=credential,
    subscription_id="31241946-ad24-4495-85e2-31248c60d164",
    resource_group_name="ariefinternal",
    workspace_name="ariefmlworkspace"
)

Register and upload the car dataset


In [7]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
my_path = "abfss://amex@ariefgen2.dfs.core.windows.net/data/mlcardata"
car_cleaned_data = Data(
    path="../../data/good_car_data.csv",
    type=AssetTypes.URI_FILE,
    description="Good Car Data",
    name="car",
    version='1'
)

ml_client.data.create_or_update(car_cleaned_data)

KeyboardInterrupt: 